In [34]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from tqdm import tqdm

BATCH_SIZE = 128

In [13]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev = 0.01))

def get_model(X, w_h1, w_h2, w_o, p_keep_input, p_keep_hidden):
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h1))
    
    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))
    
    h2 = tf.nn.dropout(h2, p_keep_hidden)
    
    return tf.matmul(h2, w_o)


mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
#dropout->Relu(matmul) -> dropout -> Relu(matmul) -> dropout -> matmul

In [35]:
X = tf.placeholder(dtype = "float", shape = [None, 784])
Y = tf.placeholder(dtype = "float", shape = [None, 10])


w_h1 = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

z = get_model(X = X, w_h1 = w_h1, w_h2 = w_h2, w_o = w_o, 
              p_keep_input = p_keep_input, p_keep_hidden = p_keep_hidden)


loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = Y, logits=z)
cost = tf.reduce_mean(loss)

train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)
predict_op = tf.argmax(z, axis = 1)

In [36]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for i in tqdm(range(100)):
        for start, end in zip(range(0, len(trX) - BATCH_SIZE, BATCH_SIZE), 
                              range(BATCH_SIZE, len(trX), BATCH_SIZE)):
        
            batch_Y = trY[start:end]
            batch_X = trX[start:end]
            
            pred = sess.run(z, feed_dict = {X:batch_X, 
                                            p_keep_input: 0.8, 
                                            p_keep_hidden: 0.5})
            
            
            sess.run(train_op, feed_dict = {Y: batch_Y, 
                                            X: batch_X, 
                                            z: pred, 
                                            p_keep_input: 0.8,
                                            p_keep_hidden: 0.5
                                           })
            
        if (i%10 == 0):
            pred_test = sess.run(predict_op, feed_dict = {X:teX, 
                                        p_keep_input: 0.8, 
                                        p_keep_hidden: 0.5})

            #cost_loss = sess.run(cost, feed_dict = {Y: teY, z: pred_test})
            accuracy = np.mean(pred_test == pred_test)
            print("Step {}, accuracy {}".format(i, accuracy))

  1%|          | 1/100 [00:07<12:30,  7.58s/it]

Step 0, accuracy 1.0


 11%|█         | 11/100 [01:23<11:17,  7.61s/it]

Step 10, accuracy 1.0


KeyboardInterrupt: 